# Topic Models

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/wine.csv', encoding='utf8')
len(df)

104637

In [3]:
df.head()

,description,country,province,variety,description_cleaned
0,This tremendous 100% varietal wine hails from ...,US,California,Cabernet Sauvignon,tremendous varietal wine hail be age year oak ...
1,Mac Watson honors the memory of a wine once ma...,US,California,Sauvignon Blanc,honor memory wine once make his mother tremend...
2,"This spent 20 months in 30% new French oak, an...",US,Oregon,Pinot Noir,spend month new french oak incorporate fruit v...
3,This re-named vineyard was formerly bottled as...,US,Oregon,Pinot Noir,re name vineyard be formerly bottle will find ...
4,The producer sources from two blocks of the vi...,US,California,Pinot Noir,producer source block vineyard wine high eleva...


Here is how the data was cleaned

In [4]:
# import spacy
# nlp = spacy.load('en')
# def clean(text):
#     return ' '.join([token.lemma_ 
#             for token in nlp(text) 
#             if token.pos_ in {'NOUN', 'VERB', 'ADJ', 'ADV', 'X'}])

In [5]:
# remove empty entries
df = df[df['description_cleaned'].notnull()]
len(df)

104637

## LDA

In [6]:
from gensim.models import LdaMulticore, TfidfModel, CoherenceModel
from gensim.corpora import Dictionary
import time # to know how long training took
import multiprocessing # to speed things up by parallelizing

In [ ]:
limit=50000

# get dictionary
df['description_cleaned'] = df.description_cleaned.apply(str)
# run on 50000 instances
instances = df.description_cleaned.apply(str.split)[:limit]
print("creating dictionary", flush=True)
# read in instances and create Dictionary object w information about frequencies etc. 
dictionary = Dictionary(instances)
# get rid of words that are too rare or too frequent
dictionary.filter_extremes(no_below=50, no_above=0.3)
print(dictionary, flush=True)

In [ ]:
#replace words by their numerical IDs and their frequency
print("translating corpus to IDs", flush=True)
ldacorpus = [dictionary.doc2bow(text) for text in instances]
# learn TFIDF values from corpus
print("tf-idf transformation", flush=True)
tfidfmodel = TfidfModel(ldacorpus)
# transform raw frequencies into TFIDF
model_corpus = tfidfmodel[ldacorpus]

In [ ]:
print(instances[0])
print(ldacorpus[0]) 
print(model_corpus[0])

## Choosing the number of topics

In [ ]:
coherence_values = []

dev_size = 10000
eval_size = 5000

for num_topics in range(5, 16):
    model = LdaMulticore(corpus=model_corpus[:dev_size], 
                         id2word=dictionary, 
                         num_topics=num_topics)

    coherencemodel_umass = CoherenceModel(model=model, 
                                          texts=instances[dev_size:dev_size+eval_size], 
                                          dictionary=dictionary, 
                                          coherence='u_mass')

    coherencemodel_cv = CoherenceModel(model=model, 
                                       texts=instances[dev_size:dev_size+eval_size], 
                                       dictionary=dictionary, 
                                       coherence='c_v')

    umass_score = coherencemodel_umass.get_coherence()
    cv_score = coherencemodel_cv.get_coherence()
    
    print(num_topics, umass_score, cv_score)
    coherence_values.append((num_topics, umass_score, cv_score))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context('poster') # use large font


scores = pd.DataFrame(coherence_values, columns=['num_topics', 'UMass', 'CV'])
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
scores.plot.line(x='num_topics', y='UMass', ax=ax[0], xticks=range(5,21));
scores.plot.line(x='num_topics', y='CV', ax=ax[1], xticks=range(5,21));


In [ ]:
num_topics = 11

# find chunksize to make about 200 updates
num_passes = 10
chunk_size = len(model_corpus) * num_passes/200
print(chunk_size)

start = time.time()
print("fitting model", flush=True)
model = LdaMulticore(num_topics=num_topics, # number of topics
                     corpus=model_corpus, # what to train on 
                     id2word=dictionary, # mapping from IDs to words
                     workers=min(10, multiprocessing.cpu_count()-1), # choose 10 cores, or whatever computer has
                     passes=num_passes, # make this many passes over data
                     chunksize=chunk_size, # update after this many instances
                     alpha=0.5
                    )
    
print("done in {}".format(time.time()-start), flush=True)


In [ ]:
# transform the data into topic distros
topic_corpus = model[model_corpus]

topic_corpus[0]

In [ ]:
model.print_topics()

In [ ]:
import re

# get the topic descritions
topic_sep = re.compile(r"0\.[0-9]{3}\*") # getting rid of useless formatting
# extract a list of tuples with topic number and descriptors from the model
model_topics = [(topic_no, re.sub(topic_sep, '', model_topic).split(' + ')) for topic_no, model_topic in
                model.print_topics(num_topics=num_topics, num_words=5)]

descriptors = []
for i, m in model_topics:
    print(i+1, ", ".join(m[:5]))
    descriptors.append(", ".join(m[:2]).replace('"', ''))

## Aggregating topics by a dependent variable

In [ ]:
target_category = 'country'
# get a list of all the topic scores for each document
scores = [[t[1] for t in topic_corpus[entry]] for entry in range(limit)]
# turn that into a data frame with N rows and K columns, each with the score of the corresponding topic
topic_distros = pd.DataFrame(data=scores, columns=descriptors)
# add the review category of each document (so we can aggregate)
topic_distros['category'] = df[target_category][:limit]

In [ ]:
topic_distros.head()

In [ ]:
import matplotlib.pyplot as plt # make graphs
import seaborn # make prettier graphs

seaborn.set_context('poster') # use large font

fig, ax = plt.subplots(figsize=(20, 10)) # set graph size
# aggregate topics by categories
aggregate_by_category = topic_distros[topic_distros.category.isin('Germany US Italy France Spain'.split())]
aggregate_by_category = aggregate_by_category.groupby(aggregate_by_category.category).mean()
# plot the graph
aggregate_by_category[descriptors].plot.bar(ax=ax);
# move the legend out
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5));

# Author Topic Model

In [ ]:
from gensim.models import AuthorTopicModel
from gensim.test.utils import datapath, temporary_file

In [ ]:
from collections import defaultdict
author2doc = defaultdict(list)

for i, country in enumerate(df.country[:limit]):
    author2doc[country].append(i)
    
len(author2doc)

In [ ]:
coherence_values = []
author_model_list = []

dev_size = 10000
eval_size = 10000

dev_author2doc = {key: [idx for idx in value if idx < dev_size] for key, value in author2doc.items()}

for num_topics in range(5, 16):
    author_model = AuthorTopicModel(corpus=list(model_corpus[:dev_size]), 
                                    author2doc=dev_author2doc, 
                                    id2word=dictionary, 
                                    num_topics=num_topics)
#     author_model_list.append(author_model)
    
    coherencemodel_umass = CoherenceModel(model=author_model, 
                                          texts=instances[dev_size:dev_size+eval_size], 
                                          dictionary=dictionary, 
                                          coherence='u_mass')

    coherencemodel_cv = CoherenceModel(model=author_model, 
                                       texts=instances[dev_size:dev_size+eval_size], 
                                       dictionary=dictionary, 
                                       coherence='c_v')

    umass_score = coherencemodel_umass.get_coherence()
    cv_score = coherencemodel_cv.get_coherence()
    
    print(num_topics, umass_score, cv_score)
    coherence_values.append((num_topics, umass_score, cv_score))

In [ ]:
%matplotlib inline
scores = pd.DataFrame(coherence_values, columns=['num_topics', 'UMass', 'CV'])
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
scores.plot.line(x='num_topics', y='UMass', ax=ax[0], xticks=range(5,16));
scores.plot.line(x='num_topics', y='CV', ax=ax[1], xticks=range(5,16));

In [ ]:
n_topics_author = 8


author_model = AuthorTopicModel(corpus=list(model_corpus), 
                                author2doc=author2doc, 
                                id2word=dictionary, 
                                num_topics=n_topics_author,
                                passes=num_passes,
                                chunksize=chunk_size,
                                alpha=0.5
)


In [ ]:
# extract a list of tuples with topic number and descriptors from the model
author_model_topics = [(topic_no, re.sub(topic_sep, '', model_topic).split(' + ')) for topic_no, model_topic in
                author_model.print_topics(num_topics=n_topics_author, num_words=5)]

author_descriptors = []
for i, m in sorted(author_model_topics):
    print(i+1, ", ".join(m[:5]))
    author_descriptors.append(", ".join(m[:2]).replace('"', ''))

In [ ]:
author_vecs = {author: {author_descriptors[t]: 0.0
                         for t in range(author_model.num_topics)}
              for author in author_model.id2author.values()
              }
for author in author_model.id2author.values():
    for (t, v) in author_model.get_author_topics(author):
        author_vecs[author][author_descriptors[t]] = v

for country in 'Germany US Italy France Spain'.split():
    print(country, author_vecs[country])        

In [ ]:
author_df = pd.DataFrame.from_dict(author_vecs)
fig, ax = plt.subplots(figsize=(20,10))
author_df['Germany US Italy France Spain'.split()].T.plot.bar(ax=ax)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5));


# Guided LDA

In [ ]:
import numpy as np
import guidedlda
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vectorize input
vectorizer = CountVectorizer(analyzer='word', 
                             ngram_range=(1,2), 
                             min_df=100, 
                             max_df=0.3, 
                             stop_words='english')

X = vectorizer.fit_transform(df.description_cleaned[:limit].tolist())

# store lookup structures for convenience
vocab = vectorizer.get_feature_names()
word2id = dict((v, idx) for idx, v in enumerate(vocab))

print(X.shape)

In [ ]:
# define topic seeds based on intuition
indicators = {
    'BRIGHT': ["fruity", "crisp", "bright"],
    'SPICE': ["licorice", "pepper", 'spice'], 
    'GREEN FRUIT': ["apple", "lemon", "citrus", "peach", "pear"],
    'DARK': ["vanilla", "smoke", "leather"],
    'RED FRUIT': ["strawberry", "raspberry", 'cherries'],
    'FOOD': ["pair", "food", "steak"],
    'FULL': ["bodied", "smoke", "medium", "vanilla"],
    'AGEING': ["age", "year", "structure"]
}

topic_names, seed_topic_list = zip(*indicators.items())
# filter out all words not actually in vocab
seed_topic_list = [[w for w in words if w in set(vocab)] for words in seed_topic_list]
    
# create a mapping {word_id: k}, which we need for the model
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id
        
print(seed_topic_list)

In [ ]:
seed_topics

In [ ]:
# define model
model = guidedlda.GuidedLDA(n_topics=len(seed_topic_list), 
                            n_iter=1000, 
                            random_state=7, 
                            refresh=50, 
                            alpha=0.5, 
                            eta=0.000001)

# fit the model with seeds
doc_topic = model.fit_transform(X, seed_topics=seed_topics, seed_confidence=10)

# retrieve the word descriptors
n_top_words = 5
topic_word = model.topic_word_
descriptors_guided = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: "{}"'.format(topic_names[i], '" "'.join(topic_words)))
    descriptors_guided.append(' '.join(topic_words[:2]))

In [ ]:
# get a list of all the topic scores for each document
# scores = [[t[1] for t in topic_corpus[entry]] for entry in range(limit)]
# turn that into a data frame with N rows and K columns, each with the score of the corresponding topic
topic_distros_guided = pd.DataFrame(data=model.doc_topic_, columns=descriptors_guided)
# add the review category of each document (so we can aggregate)
topic_distros_guided['category'] = df[target_category][:limit]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10)) # set graph size
# aggregate topics by review categories
aggregate_by_category = topic_distros_guided[topic_distros_guided.category.isin('Germany US Italy France Spain'.split())]
aggregate_by_category = aggregate_by_category.groupby(aggregate_by_category.category).mean()

# plot the graph
aggregate_by_category[descriptors_guided].plot.bar(ax=ax);
# move the legend out
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5));